In [ ]:
import imdlib as imd
import rioxarray
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
#import geopandas as gpd
import glob
import os

In [ ]:
# !pip install imdlib
# !pip install rioxarray
# !pip install xshape

In [ ]:
#Data download

# start_yr = 1990
# end_yr = 2022
# variable = 'rain' # other options are ('tmin'/ 'tmax')
# data = imd.get_data(variable, start_yr, end_yr, fn_format='yearwise')

In [ ]:
#temporal bounds
# start_yr = 1990
# end_yr = 2022
# variable = 'rain' # other options are ('tmin'/ 'tmax')
# # file_dir = (r'./datasets/') #Path to the saved files
# data_y90y22 = imd.open_data(variable, start_yr, end_yr,'yearwise', data_dir)

# rf = data_y90y22.get_xarray()
# rf = rf.where(rf['rain'] != -999.)

In [ ]:
rf

In [ ]:
# x=rf.sel(lat=slice(22,29),lon=slice(75,90))
# x.rio.write_crs(4326, inplace=True)
# x.to_netcdf("GangaPlains_rain_1990_2022.nc") 

In [ ]:
ds = xr.open_dataset('datasets/GangaPlains_rain_1990_2022.nc')

In [ ]:
ds = ds.where(ds['rain'] != -999.) #Remove NaN values
ds['rain'].mean('time').plot()

In [ ]:
ds.rain.mean(dim=('lon','lat')).plot()

In [ ]:
ds.rain.isel(time=0).plot()

In [ ]:
ds_y = ds.rain.groupby('time.year').sum() #.where((ds['rain'] > 100) & (ds['rain'] < 200))

In [ ]:
y_plot = ds_y.sel(lat=slice(24.5,26.5),lon=slice(85,86.5)).where(ds_y>0).plot(x='lon',y='lat',col='year', col_wrap=3)

In [ ]:
ds.rain.sel(lat=25.25,lon=85.5,time=slice('1990-01-01','2022-01-01')).groupby('time.dayofyear').mean().plot()

# ds.rain.sel(lat=25.25,lon=85.5,time=slice('1990-01-01','2022-01-01')).groupby('time.weekofyear').mean().plot()

# ds.rain.sel(lat=25.25,lon=85.5,time=slice('1990-01-01','2022-01-01')).groupby('time.month').mean().plot()

In [ ]:
ds.rain.sel(lat=25.25,lon=85.5,time=slice('1990-01-01','2000-01-01')).groupby(
    'time.dayofyear').mean().plot(label='1st epoch', color='k')
ds.rain.sel(lat=25.25,lon=85.5,time=slice('2001-01-01','2021-01-01')).groupby(
    'time.dayofyear').mean().plot(label='2nd epoch', color='r')

In [ ]:
df = ds_y.sel(lat=26,lon=86,method='nearest').to_dataframe()

df.to_csv('df_y.csv')

In [ ]:
df

In [ ]:
x=ds.sel(lat=slice(22,28),lon=slice(83,87))
x['rain'].max('time').plot()

# Monsoonal RF

In [ ]:
ds_monsoon = ds.sel(time=ds.time.dt.month.isin([6,7,8,9])).groupby('time.year').sum()

In [ ]:
ds_monsoon_mean = ds_monsoon.rain.mean('year')
ds_monsoon_mean = ds_monsoon_mean.where(ds_monsoon_mean!= 0.) 

ds_monsoon_mean.plot()

In [ ]:
#Save mean monsoonal RF as geoTiff
#ds_monsoon_mean.rename({'lat':'y','lon':'x'}).rio.set_crs(4326,inplace=True).rio.to_raster('mean_monRF_1990_2020.tif')

In [ ]:
roll = x.rolling(time=30, center=True).mean()
x.rain.sel(lat=26,lon=86).plot(label='rf', color='r')
roll.rain.sel(lat=26,lon=86).plot(label='monthly rolling mean', color='k')
plt.legend()

In [ ]:
#For removing longterm mean monthly
def remove_time_mean(x):
    return x - x.mean(dim='time')

ds_anom = ds.groupby('time.month').apply(remove_time_mean)
ds_anom

In [ ]:
# Import required packages
%matplotlib inline
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import h5py
import xarray as xr
import rioxarray 
import datetime as dt
from xarrayutils.utils import linear_trend, xr_linregress, xr_detrend
import hvplot.xarray
import hvplot.pandas
import holoviews as hv

import cartopy.crs as ccrs
from xrspatial.classify import natural_breaks

In [ ]:
xDtrend = xr_detrend(x.rain)
# xDtrend

In [ ]:
x.rain.sel(latitude=26,longitude=86,time=slice('1999-01-01','1999-08-01')).plot(label='rf', color='r')
xDtrend.sel(latitude=26,longitude=86,time=slice('1999-01-01','1999-08-01')).plot(label='rf', color='g')


In [ ]:
x = x.rename({'lat':'latitude','lon':'longitude'})
x

In [ ]:
#Calculate linear trends using xarrayutils 
x_LTrend = linear_trend(x.rain, 'time')
x_LTrend

In [ ]:
x_LTrend.slope.where(x_LTrend.p_value<0.05).hvplot(geo=True)